In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [4]:
%cd S:\PPMI_RNA\Processed\Progression_Analysis\Male\HYS_1vs2vs3
data = pd.read_csv('CountData_HYS1vs2vs3_PD_withHYS_Male.csv')
data = data[data['HYS'] != 3]
data.head(1)

S:\PPMI_RNA\Processed\Progression_Analysis\Male\HYS_1vs2vs3


,HudAlphaID,HYS,ENSG00000223972.5,ENSG00000227232.5,ENSG00000278267.1,ENSG00000243485.5,ENSG00000284332.1,ENSG00000237613.2,ENSG00000268020.3,ENSG00000240361.2,...,ENSG00000273739.1,ENSG00000276700.1,ENSG00000276312.1,ENSG00000275757.1,ENSG00000278573.1,ENSG00000276017.1,ENSG00000278817.1,ENSG00000277196.4,ENSG00000278625.1,ENSG00000277374.1
0,5104_SL_0001,2,15,232,15,0,0,0,0,0,...,0,1,16,1,0,2,19,0,0,0


In [5]:
X = data.drop(['HudAlphaID', 'HYS'], axis = 1)
y = data['HYS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   ENSG00000223972.5  ENSG00000227232.5  ENSG00000278267.1  ENSG00000243485.5  \
0                 15                232                 15                  0   
1                  0                 97                  8                  0   

   ENSG00000284332.1  ENSG00000237613.2  ENSG00000268020.3  ENSG00000240361.2  \
0                  0                  0                  0                  0   
1                  0                  0                  0                  0   

   ENSG00000186092.6  ENSG00000238009.6  ...  ENSG00000273739.1  \
0                  0                124  ...                  0   
1                  2                124  ...                  0   

   ENSG00000276700.1  ENSG00000276312.1  ENSG00000275757.1  ENSG00000278573.1  \
0                  1                 16                  1                  0   
1                  9                  0                  1                  0   

   ENSG00000276017.1  ENSG00000278817.1  ENSG00000277196.4  ENSG00

In [6]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [7]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7191780821917808

In [8]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100


BorutaPy finished running.

Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	0
Rejected: 	58780


BorutaPy(estimator=RandomForestClassifier(n_estimators=3428,
                                          random_state=RandomState(MT19937) at 0x27137617140),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x27137617140, verbose=1)

In [9]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  0


In [10]:
selected_rf_features = pd.DataFrame({'GeneID':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [11]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,CpG,Ranking
6751,ENSG00000125551.18,2
25860,ENSG00000254859.1,3
40171,ENSG00000211935.3,4


### Top 50

In [12]:
Top50.to_csv ('HYS1vs2_Boruta_Top50_Male.csv', index = False)
selected_rf_features.to_csv ('HYS1vs2_Boruta_All_Male.csv', index = False)

In [13]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top50_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_7424\1143389863.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000186891.13,ENSG00000173372.16,ENSG00000168765.16,ENSG00000125551.18,ENSG00000229754.1,ENSG00000124019.9,ENSG00000206561.12,ENSG00000179934.6,ENSG00000012223.12,ENSG00000163661.3,...,ENSG00000211966.2,ENSG00000211970.3,ENSG00000062524.15,ENSG00000103148.15,ENSG00000278987.1,ENSG00000267265.5,ENSG00000160307.9,ENSG00000278196.3,ENSG00000100336.17,HYS
0,138,246,404,150,808,133,406,98,708,66,...,138,14,242,3378,177,149,49,248,61,2
1,218,165,309,118,169,127,322,89,258,59,...,54,11,179,1456,100,107,41,15,62,2


### Top 100

In [14]:
Top100 = selected_rf_features.iloc[:100, :]
Top100.to_csv ('HYS1vs2_Boruta_Top100_Male.csv', index = False)
Top100.head(2)

,CpG,Ranking
6751,ENSG00000125551.18,2
25860,ENSG00000254859.1,3


In [15]:
# create a list of column names to subset from data
col_names = Top100.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top100_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_7424\3681904022.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000186891.13,ENSG00000173372.16,ENSG00000122420.9,ENSG00000168765.16,ENSG00000125551.18,ENSG00000144290.16,ENSG00000115457.9,ENSG00000229754.1,ENSG00000124019.9,ENSG00000206561.12,...,ENSG00000211638.2,ENSG00000278196.3,ENSG00000211673.2,ENSG00000218537.1,ENSG00000273295.1,ENSG00000100336.17,ENSG00000275975.1,ENSG00000182489.8,ENSG00000260585.1,HYS
0,138,246,44,404,150,447,39,808,133,406,...,183,248,280,83,505,61,25,83,5,2
1,218,165,6,309,118,157,11,169,127,322,...,13,15,21,85,320,62,15,120,6,2


### Top 250

In [16]:
Top250 = selected_rf_features.iloc[:250, :]
Top250.to_csv ('HYS1vs2_Boruta_Top250_Male.csv', index = False)
Top250.head(2)

,CpG,Ranking
6751,ENSG00000125551.18,2
25860,ENSG00000254859.1,3


In [17]:
# create a list of column names to subset from data
col_names = Top250.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top250_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_7424\2749445622.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000186891.13,ENSG00000049247.13,ENSG00000132879.13,ENSG00000285646.1,ENSG00000173372.16,ENSG00000213172.3,ENSG00000117399.13,ENSG00000122420.9,ENSG00000238015.2,ENSG00000137941.16,...,ENSG00000133477.16,ENSG00000216866.5,ENSG00000275975.1,ENSG00000186310.9,ENSG00000182489.8,ENSG00000260585.1,ENSG00000129682.15,ENSG00000155966.13,ENSG00000185010.14,HYS
0,138,37,504,6,246,77,82,44,31,93,...,84,61,25,86,83,5,27,318,215,2
1,218,46,206,3,165,39,6,6,26,133,...,87,52,15,65,120,6,37,289,135,2


### Top 500

In [18]:
Top500 = selected_rf_features.iloc[:500, :]
Top500.to_csv ('HYS1vs2_Boruta_Top500_Male.csv', index = False)
Top500.head(2)

,CpG,Ranking
6751,ENSG00000125551.18,2
25860,ENSG00000254859.1,3


In [19]:
# create a list of column names to subset from data
col_names = Top500.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['HYS'] = data['HYS']
df_subset.to_csv ('HYS1vs2_Boruta_Top500_Data_Male.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_7424\987567329.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['HYS'] = data['HYS']


,ENSG00000278267.1,ENSG00000186891.13,ENSG00000235169.8,ENSG00000049247.13,ENSG00000132879.13,ENSG00000175206.10,ENSG00000285646.1,ENSG00000187942.11,ENSG00000173372.16,ENSG00000284602.1,...,ENSG00000186310.9,ENSG00000182489.8,ENSG00000250995.1,ENSG00000260585.1,ENSG00000221930.6,ENSG00000129682.15,ENSG00000155966.13,ENSG00000198883.11,ENSG00000185010.14,HYS
0,15,138,206,37,504,0,6,3,246,396,...,86,83,156,5,19,27,318,22,215,2
1,8,218,66,46,206,3,3,19,165,263,...,65,120,138,6,48,37,289,10,135,2
